# Uso de funciones

Los asistentes nos permiten definir llamadas a funciones de manera que podemos entremezclar nuestro código (independientemente del lenguaje que use este) con la interacción con asistentes.

Como siempre empezamos inicializando el cliente y creando nuestro asistente:

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [2]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ"
openai.api_key = os.environ["OPENAI_API_KEY"]
client = openai.OpenAI()

In [3]:
instrucciones_asistente = """
Eres un profesor de Python para principiantes. Los usuarios te preguntarán sobre
temas relacionados con programación y tú debes ayudarlos. Para ello les explicarás
brevemente la teoría detrás de los conceptos implicados, la sintaxis general para
resolver la duda y por último darás un ejemplo de código comentando en cada línea
de código lo que estás haciendo.
"""

In [4]:
asistente_funciones = client.beta.assistants.create(
  name="Profe de Python vfunc",
  description="Versión mejorada incluyendo llamadas a funciones",
  instructions=instrucciones_asistente,
  model="gpt-4o"
)

A la hora de definir una función usaremos un diccionario en el que iremos incluyendo la información necesaria:

In [5]:
funcion_documentacion = {
    "name" : "longitud_itinerarios",
    "description" : "Estima el tiempo que tardará en completar el itinerario un estudiante promedio",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "itinerario" : {
                "type" : "string",
                "description" : "Nombre del itinerario"
            },
            "unit" : {
                "type" : "string",
                "enum" : ["días"],
                "description": "Unidad de medida de la duración del itinerario"
            }
        },
        "required" : ["itinerario"]
    }
}

In [6]:
asistente_funciones = client.beta.assistants.update(
  assistant_id=asistente_funciones.id,
  tools = [
      {
          "type" : "function",
          "function" : funcion_documentacion
      }
  ]
)

In [7]:
def longitud_itinerarios(itinerario):
  if itinerario == "satélite":
    return "3"
  elif itinerario == "planeta":
    return "6"
  elif itinerario == "estrella":
    return "9"
  else:
    return "El itinerario indicado no existe"

In [8]:
longitud_itinerarios("estrella")

'9'

In [9]:
hilo = client.beta.threads.create()
mensaje = client.beta.threads.messages.create(
  thread_id=hilo.id,
  role="user",
  content="¿Cuánto tardaré en completar el itinerario estrella?¿Y mi hermano Nico el itinerario satélite?",
)

In [10]:
ejecucion = client.beta.threads.runs.create_and_poll(
  thread_id=hilo.id,
  assistant_id=asistente_funciones.id,
)

In [11]:
ejecucion.status

'requires_action'

El asistente ha detectado la llamada a la función y está pendiente de que le enviemos la información:

In [12]:
ejecucion.required_action.submit_tool_outputs.tool_calls

[RequiredActionFunctionToolCall(id='call_LIRLMX7TZQW9RALmggZg19Cm', function=Function(arguments='{"itinerario": "estrella"}', name='longitud_itinerarios'), type='function'),
 RequiredActionFunctionToolCall(id='call_QbFk8YpgLvqfWD3W85l9DT8p', function=Function(arguments='{"itinerario": "satélite"}', name='longitud_itinerarios'), type='function')]

In [13]:
primera_llamada_herramienta = ejecucion.required_action.submit_tool_outputs.tool_calls[0]
primera_llamada_herramienta

RequiredActionFunctionToolCall(id='call_LIRLMX7TZQW9RALmggZg19Cm', function=Function(arguments='{"itinerario": "estrella"}', name='longitud_itinerarios'), type='function')

In [14]:
eval(primera_llamada_herramienta.function.arguments)["itinerario"]

'estrella'

In [15]:
def extrae_argumentos(llamada_herramienta, nombre_argumento):
  argumento = eval(llamada_herramienta.function.arguments)[nombre_argumento]
  return argumento


In [16]:
extrae_argumentos(primera_llamada_herramienta, "itinerario")

'estrella'

In [17]:
extrae_argumentos(ejecucion.required_action.submit_tool_outputs.tool_calls[1], "itinerario")

'satélite'

Almacenamos en una lista los resultados que necesitamos enviarle al usuario. Tras ello, recorremos las llamadas a las herramientas y vamos llamando a nuestra función y almacenando los resultados:

In [18]:
salidas_herramienta = []
for llamada_herramienta in ejecucion.required_action.submit_tool_outputs.tool_calls:
  if llamada_herramienta.function.name == "longitud_itinerarios":
    argumento_funcion = extrae_argumentos(llamada_herramienta, "itinerario")
    resultado_funcion = longitud_itinerarios(argumento_funcion)
    salidas_herramienta.append({
      "tool_call_id": llamada_herramienta.id,
      "output": resultado_funcion
    })
salidas_herramienta

[{'tool_call_id': 'call_LIRLMX7TZQW9RALmggZg19Cm', 'output': '9'},
 {'tool_call_id': 'call_QbFk8YpgLvqfWD3W85l9DT8p', 'output': '3'}]

Finalmente enviamos las respuesta a la herramienta para que pueda devolver la salida y recuperamos los mensajes.

In [19]:
if salidas_herramienta:
  try:
    ejecucion_envio = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=hilo.id,
      run_id=ejecucion.id,
      tool_outputs=salidas_herramienta
    )
    print("Resultados enviados correctamente.")
  except Exception as e:
    print("Ha habido un problema enviando los resultados:", e)
else:
  print("No hay resultados para enviar.")

if ejecucion_envio.status == 'completed':
  mensajes = client.beta.threads.messages.list(
    thread_id=hilo.id
  )
  print(mensajes)
else:
  print(ejecucion_envio.status)

Resultados enviados correctamente.
SyncCursorPage[Message](data=[Message(id='msg_ZqS85ZquxLftsrFbryMgvg3j', assistant_id='asst_8iyZsO7QOYeF309uHPWsCZiw', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Se estima que completarás el itinerario estrella en 9 días, mientras que tu hermano Nico completará el itinerario satélite en 3 días.'), type='text')], created_at=1718213937, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_vBpNvH6FJSWZJyvFssyDQTjs', status=None, thread_id='thread_bFE5XgxqIiE5dpyFZv9OisMM'), Message(id='msg_KQNoRkLqovBFpMpzCvlrGp1Z', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='¿Cuánto tardaré en completar el itinerario estrella?¿Y mi hermano Nico el itinerario satélite?'), type='text')], created_at=1718213611, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', rol

In [20]:
def recupera_conversacion_de_hilo(hilo):
  conversacion = {}
  mensajes_hilo = client.beta.threads.messages.list(hilo)
  longitud_hilo = len(mensajes_hilo.data)
  for indice in range(0, longitud_hilo):
    informacion_mensaje = mensajes_hilo.data[longitud_hilo-1-indice]
    conversacion[indice] = (informacion_mensaje.role, informacion_mensaje.content[0].text.value)

  return conversacion

conversacion_actualizada = recupera_conversacion_de_hilo(hilo.id)
conversacion_actualizada

{0: ('user',
  '¿Cuánto tardaré en completar el itinerario estrella?¿Y mi hermano Nico el itinerario satélite?'),
 1: ('assistant',
  'Se estima que completarás el itinerario estrella en 9 días, mientras que tu hermano Nico completará el itinerario satélite en 3 días.')}

## Cierre

En este cuaderno hemos visto un ejemplo muy sencillo de cómo utilizar funciones definidas por fuera de los asistentes dentro del flujo conversacional. Es importante entender que esto aporta una gran cantidad de opciones como por ejemplo, la llamada a otras API's para el cruce de información interna y externa.